In [ ]:
import PyPDF2
import re

resume_path = '/Users/sakshigupta/Desktop/FYP/AI-RecruitmentTool/back-end/data/sample.pdf'
job_description = '/Users/sakshigupta/Desktop/FYP/AI-RecruitmentTool/back-end/data/job_description.txt'


def preprocess_text(text):
    # Convert to lowercase and remove special characters
    text = re.sub(r'[^\w\s]', '', text.lower())
    return text
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

def extract_data_from_text(file_path) :
    file_contents =''
    with open(file_path, 'r') as file:
        file_contents = file.read()
    return file_contents


resume_text = extract_text_from_pdf(resume_path)
job_description_text = extract_data_from_text(job_description)
processed_resume = preprocess_text(resume_text)
processed_job_description = preprocess_text(job_description_text)

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


vectorizer = TfidfVectorizer()
count_matrix=vectorizer.fit_transform([processed_resume,processed_job_description])
MatchPercentage=cosine_similarity(count_matrix)[0][1]*100
print(f'Match Percentage is: {MatchPercentage:.2f}')


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = CountVectorizer()
count_matrix=vectorizer.fit_transform([processed_resume,processed_job_description])
MatchPercentage=cosine_similarity(count_matrix)[0][1]*100
print(f'Match Percentage is: {MatchPercentage:.2f}')

In [ ]:


# !!! Kernel crashing on local - added code in google collab
# https://colab.research.google.com/drive/1kOPN6OPJqdJAif8ClZcQnUcsRBn2z_Ag?usp=sharing


# from sentence_transformers import SentenceTransformer
# def sentence_bert_cosine_similarity(resume, job_description):
#     model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    
#     # Ensure inputs are lists
#     resumes = [resume] if isinstance(resume, str) else resume
#     job_descriptions = [job_description] if isinstance(job_description, str) else job_description
    
#     document_embeddings = model.encode(resumes + job_descriptions)
    
#     resume_embeddings = document_embeddings[:len(resumes)]
#     job_embeddings = document_embeddings[len(resumes):]
    
#     return cosine_similarity(resume_embeddings, job_embeddings)


# sentence_bert_similarities = sentence_bert_cosine_similarity(processed_resume, processed_job_description)
# print(f"Sentence-BERT similarity: {sentence_bert_similarities[0][0]:.4f}")

In [ ]:
import os
import base64
from pymongo import MongoClient

# MongoDB connection
client = MongoClient('mongodb://localhost:27017/')
db = client['AI-RECRUITMENT']
collection = db['CANDIDATES']

# Function to encode DOCX file to base64
def encode_docx_to_base64(docx_path):
    with open(docx_path, "rb") as docx_file:
        encoded_string = base64.b64encode(docx_file.read()).decode('utf-8')
    return encoded_string

# Path to the directory containing the DOCX files
docx_directory = '/Users/sakshigupta/Downloads/Resumes'

# Iterate over all DOCX files in the directory
for docx_file in os.listdir(docx_directory):
    if docx_file.endswith('.docx'):
        docx_path = os.path.join(docx_directory, docx_file)
        candidate_name = os.path.splitext(docx_file)[0]
        encoded_docx = encode_docx_to_base64(docx_path)
        
        # Prepare the document for MongoDB
        document = {
            "name": candidate_name,
            "resume": encoded_docx,
            "cover_letter": encoded_docx
        }
        
        # Insert the document into MongoDB
        collection.insert_one(document)
        print(f"Inserted {candidate_name}'s resume into MongoDB")

print("All DOCX files have been processed and inserted into MongoDB.")
